# METODOS DE RECUPERACIÓN DE DATOS, UNO, MULTIPLES, O TODOS.

| Base de Datos | Conector                             | `fetchone`                                     | `fetchmany`                                       | `fetchall`                                 |
|---------------|--------------------------------------|-------------------------------------------------|----------------------------------------------------|--------------------------------------------|
| MySQL         | mysql-connector-python o pymysql     | `unique_row = cursor.fetchone()`                | `many_rows = cursor.fetchmany(size)`               | `all_rows = cursor.fetchall()`               |
| PostgreSQL    | psycopg2                              | `unique_row = cursor.fetchone()`                | `many_rows = cursor.fetchmany(size)`               | `all_rows = cursor.fetchall()`               |
| SQL Server    | pyodbc                               | `unique_row = cursor.fetchone()`                | `many_rows = cursor.fetchmany(size)`               | `all_rows = cursor.fetchall()`               |
| Oracle        | cx_Oracle                            | `unique_row = cursor.fetchone()`                | `many_rows = cursor.fetchmany(size)`               | `all_rows = cursor.fetchall()`               |
| Snowflake     | Snowflake Connector                  | `unique_row = cursor.fetchone()`                | `many_rows = cursor.fetchmany(size)`               | `all_rows = cursor.fetchall()`               |
| MongoDB       | pymongo                              | `unique_document = collection.find_one({'filtro': 'condicion'})` | `many_documents = [doc for doc in collection.find({'filtro': 'condicion'}).limit(size)]` | `all_documents = list(collection.find())` |                                |

# METODOS DE INSERCIÓN DE DATOS.

| Base de Datos | Conector                                  | Método de Inserción Masiva                                 | Ejemplo de Uso                                           |
|---------------|-------------------------------------------|-----------------------------------------------------------|-----------------------------------------------------------|
| PostgreSQL    | psycopg2                                   | `execute_values`                                          | `cursor.execute_values('nombre_tabla', registros)`         |
| MySQL         | mysql-connector-python o pymysql            | `executemany`                                             | `cursor.executemany(query, registros)`                     |
| SQL Server    | pyodbc                                    | `executemany` o `BULK INSERT` para grandes volúmenes      | `cursor.executemany(query, registros)`                     |
| Oracle        | cx_Oracle                                 | `executemany`                                             | `cursor.executemany(query, registros)`                     |
| Snowflake     | Snowflake Connector                       | Copia masiva con opciones como COPY INTO o COPY INTO (file_format) | Ver la documentación de Snowflake para la sintaxis específica |
| MongoDB       | pymongo                                   | `insert_many`                                             | `collection.insert_many(registros)`                        |

In [ ]:
from abc import ABC, abstractmethod

# Clase base para animales
class IAnimal(ABC):
    @abstractmethod
    def speak(self):
        pass

# Subclases específicas para tipos de animales
class Dog(IAnimal):
    def speak(self):
        return "Woof!"

class Cat(IAnimal):
    def speak(self):
        return "Meow!"
    
class Pato:
    def speak(self):
        return "Cuack!"

# Fábrica de animales utilizando polimorfismo
class AnimalFactory:
    def __init__(self, Animal, **kwargs):
        if not issubclass(IAnimal, ABC) or not issubclass(Animal, IAnimal):
            raise TypeError("La clase debe heredar de IAnimal")
        self.animal_class = Animal

    def create_animal(self):
        return self.animal_class()

# Uso de polimorfismo para crear animales
dog_factory = AnimalFactory(Dog)
cat_factory = AnimalFactory(Cat)
duck_factory = AnimalFactory(Pato)  # Esto generará un error en tiempo de ejecución

dog = dog_factory.create_animal()
cat = cat_factory.create_animal()
duck = duck_factory.create_animal()

# Llamada a método común sin saber la clase concreta
print(dog.speak())  # Output: Woof!
print(cat.speak())  # Output: Meow!
print(duck.speak())  # Output: Cuack!


In [ ]:
issubclass(ABC, IAnimal)

In [ ]:
from abc import ABC, abstractmethod

class IDataBaseConnector(ABC):
    @abstractmethod
    def get_connection_url(self):
        pass

class DummyConnector(IDataBaseConnector):
    def __init__(self, connection_params):
        self.connection_params = connection_params

    def get_connection_url(self):
        return f"dummy://user:{self.connection_params['user']}@{self.connection_params['host']}:{self.connection_params['port']}/database"

class DummyConnectorV2(IDataBaseConnector):
    def __init__(self, connection_params):
        self.connection_params = connection_params

    def get_connection_url(self):
        return f"dummy_v2://user:{self.connection_params['user']}@{self.connection_params['host']}:{self.connection_params['port']}/database"

class SingletonConnectionFactory:

    _instances = {}  # Utilizamos un diccionario para manejar diferentes instancias por tipo de conector

    def __new__(cls, IDataBaseConnector, connection_params):
        if not issubclass(IDataBaseConnector, IDataBaseConnector):
            raise TypeError("La clase del conector debe heredar de IDataBaseConnector")
        
        # Clave única basada en el nombre de la clase del conector
        key = IDataBaseConnector.__name__

        if key not in cls._instances:
            print(f"Nueva instancia creada para {key}")
            cls._instances[key] = super(SingletonConnectionFactory, cls).__new__(cls)
            cls._instances[key].connector = IDataBaseConnector(connection_params=connection_params)
            cls._instances[key].session = None
        else:
            print(f"La instancia para {key} ya existía, reutilizando...")

        return cls._instances[key]

    def create_session(self):
        try:
            connection_url = self.connector.get_connection_url()
            print(f"Session created with connection URL: {connection_url}")
        except Exception as error:
            print(f"Error creating session: {error}")

# Uso de SingletonConnectionFactory con DummyConnector
connector_params = {"user": "john", "host": "localhost", "port": 5432}

factory_instance_1 = SingletonConnectionFactory(DummyConnector, connector_params)
factory_instance_2 = SingletonConnectionFactory(DummyConnector, connector_params)

assert factory_instance_1 is factory_instance_2

factory_instance_1.create_session()

# Uso de SingletonConnectionFactory con DummyConnectorV2
connector_params_v2 = {"user": "jane", "host": "example.com", "port": 8080}

factory_instance_3 = SingletonConnectionFactory(DummyConnectorV2, connector_params_v2)
factory_instance_4 = SingletonConnectionFactory(DummyConnectorV2, connector_params_v2)

assert factory_instance_3 is factory_instance_4  # Ambas instancias deben ser iguales
factory_instance_3.create_session()


In [ ]:
factory_instance_4 is factory_instance_3

In [ ]:
factory_instance_2

# <center> DECORADORES </center>
### Ampliar la funcionalidad básica de una función
### Los decoradores en Python son funciones que toman otra función como argumento y la extienden, complementan o modifican su comportamiento. 
### Esto se puede utilizar para realizar tareas adicionales antes o después de la ejecución de la función principal, 
### de manera similar a cómo funcionan los interceptores en otros contextos.

- adicionar logs
- extraer datos
- medir tiempo de cuanto tarda la ejecución de la función original
- ejecutar código antes de la ejecución de la función
- ejecutar código después de la ejecución de la función
- parsear la respuesta de la función, por ejemplo convertir todo en utf8, todo con 2 decimales, todos mayusculas, etc.

In [ ]:
def decorador_2(f):
    def funcion_nueva():
        f()
        print("Funcionalidad extra 2")
    return funcion_nueva

def decorador(f):
    def funcion_nueva():
        print("Funcionalidad extra")
        f()
    return funcion_nueva

@decorador_2
@decorador
def funcion_inicial():
    print("Funcionalidad inicial")

funcion_inicial()

In [ ]:

# code for testing decorator chaining 
def decor1(func): 
    def inner(): 
        x = func() 
        return x * x 
    return inner 
 
def decor(func): 
    def inner(): 
        x = func() 
        return 2 * x 
    return inner 
 
@decor1
@decor
def num(): 
    return 10
 
@decor
@decor1
def num2():
    return 10
   
print(num()) 
print(num2())

Autenticación y Autorización: Puedes usar decoradores para agregar fácilmente lógica de autenticación y autorización a tus vistas o funciones. Esto puede hacer que tu código sea más limpio y modular.

```python
@requiere_autenticacion
@requiere_permiso("admin")
def administrar_usuario(usuario_id):
    # Lógica para administrar un usuario
    
``
Logging: Los decoradores son útiles para agregar registros a funciones o métodos sin modificar su lógica interna.
```python
@registrar_log
def operacion_critica():
    # Lógica de operación crítica
```

Métricas de Rendimiento: Puedes usar decoradores para medir el tiempo de ejecución de funciones clave.
```python
@medir_tiempo
def proceso_largo():
    # Lógica de un proceso que quieres medir
```

Caching: Puedes implementar fácilmente caching para funciones costosas.
```python
@cache
def resultado_costoso(parametro):
    # Lógica costosa
```

Validación de Formularios: En frameworks web como Django, los decoradores pueden ser útiles para validar automáticamente los formularios antes de procesar una vista.
```python
@validar_formulario(MiFormulario)
def procesar_formulario(request, formulario):
    # Lógica para procesar el formulario
```

Manejo de Transacciones: Puedes usar decoradores para gestionar automáticamente las transacciones de base de datos.
```python
@manejar_transaccion
def actualizar_base_de_datos(datos):
    # Lógica de actualización de la base de datos
```

Rutas de API en Frameworks Web: En frameworks web como Flask, los decoradores se utilizan comúnmente para definir rutas de API y agregar middleware.
```python
@app.route("/api/endpoint")
@requiere_autenticacion
def api_endpoint():
    # Lógica de la API
```

Retry de Funciones: Puedes usar decoradores para implementar lógica de reintento en funciones que pueden fallar temporalmente.
```python
@retry(intentos=3)
def operacion_inestable():
    # Lógica inestable
```

In [ ]:
def decorador_de_decoradores(funcion_a_procesar):
    def mostrar(*args, **kwargs):
        print('Inicio Proceso, el nombre de la funcion a procesar es: ', funcion_a_procesar.__name__)
        print('Inicio Proceso con parametros: ', args, kwargs)
        return funcion_a_procesar(*args, **kwargs) * 0
        
    return mostrar

@decorador_de_decoradores
def funcion_decoradora(funcion_a_procesar):
    def mostrar(*args, **kwargs):
        print('Inicio Proceso, el nombre de la funcion a procesar es: ', funcion_a_procesar.__name__)
        print('Inicio Proceso con parametros: ', args, kwargs)
        
        print('Finalizo Proceso')
        return funcion_a_procesar(*args, **kwargs) * 2
    return mostrar

@funcion_decoradora
def sumar(a, b):
    return a + b
    
sumar(1, 2)

In [ ]:
def decorador_de_decoradores(decorador):
    def wrapper(funcion_a_procesar):
        def mostrar(*args, **kwargs):
            
            print(f'Inicio Paso decorador_de_decoradores parametros:', args, kwargs)
            print('decorador_de_decoradores va a llamar a la funcion :', decorador.__name__)
            print()
            
            resultado = decorador(funcion_a_procesar)(*args, **kwargs) * 5
            print(f'Finalizo Paso decorador_de_decoradores con resultado :', resultado)
            print()
            return resultado
        return mostrar
    return wrapper

@decorador_de_decoradores
def funcion_decoradora(funcion_a_procesar):
    def wrapper(*args, **kwargs):
        print(f'Inicio Paso funcion_decoradora parametros:', args, kwargs)
        print('funcion_decoradora va a llamar a la funcion :', funcion_a_procesar.__name__)
        print()
        
        resultado = funcion_a_procesar(*args, **kwargs) * 2
        print(f'Finalizo Paso funcion_decoradora  con resultado :', resultado)
        print()
        return resultado
    return wrapper

@funcion_decoradora
def sumar(*args):
    print('Inicio Paso funcion SUMA :', args)
    print()
    return sum(args)

resultado = sumar(1, 2)
print('Resultado final:', resultado)


In [ ]:
def decorador_de_decoradores(funcion_decoradora):
    @wraps(funcion_decoradora)
    def mostrar(*args, **kwargs):
        print('Inicio Proceso, el nombre de la funcion a procesar es: ', funcion_decoradora.__name__)
        print('Inicio Proceso con parametros: ', args, kwargs)
        return funcion_decoradora(*args, **kwargs) * 5
    return mostrar
@decorador_de_decoradores

In [1]:
from abc import ABC, abstractmethod

class INotificacion(ABC):
    
    @abstractmethod
    def crear_notificacion(self, funcion, descripcion):
        pass

class NotificacionSlack(INotificacion):
    
    def crear_notificacion(self, funcion, descripcion):
        service = self.__class__.__name__.replace('Notificacion', '')
        print(f'Notificacion enviada al {service}, El proceso de la funcion {funcion} fue {descripcion}!!!')
    
class NotificacionJira(INotificacion):
    
    def crear_notificacion(self, funcion, descripcion):
        service = self.__class__.__name__.replace('Notificacion', '')
        print(f'Notificacion enviada al {service}, El proceso de la funcion {funcion} fue {descripcion}!!!')
    
class NotificacionEmail(INotificacion):
    
    def crear_notificacion(self, funcion, descripcion):
        service = self.__class__.__name__.replace('Notificacion', '')
        print(f'Notificacion enviada al {service}, El proceso de la funcion {funcion} fue {descripcion}!!!')
    
class NotificacionCloudwatch(INotificacion):
    
    def crear_notificacion(self, funcion, descripcion):
        service = self.__class__.__name__.replace('Notificacion', '')
        print(f'Notificacion enviada al {service}, El proceso de la funcion {funcion} fue {descripcion}!!!')
    
class NotificacionDataDog(INotificacion):
    
    def crear_notificacion(self, funcion, descripcion):
        service = self.__class__.__name__.replace('Notificacion', '')
        print(f'Notificacion enviada al {service}, El proceso de la funcion {funcion} fue {descripcion}!!!')

class NotificacionLog(INotificacion):
    
    def crear_notificacion(self, funcion, descripcion):
        service = self.__class__.__name__.replace('Notificacion', '')
        print(f'Notificacion enviada al {service}, El proceso de la funcion {funcion} fue {descripcion}!!!')
        
class NotificacionGrafana(INotificacion):
    
    def crear_notificacion(self, funcion, descripcion):
        service = self.__class__.__name__.replace('Notificacion', '')
        print(f'Notificacion enviada al {service}, El proceso de la funcion {funcion} fue {descripcion}!!!')

In [2]:
from functools import wraps
import time

def validar_datos(funcion_a_decorar):
    
    @wraps(funcion_a_decorar)
    def funcion_interna_que_procesa_funcion_a_decorar_y_adiciona_comportamientos(*args, **kwargs):
        print(f'Se inició la VALIDACIÓN de los datos de la función {funcion_a_decorar.__name__}')
        
        # Verifica que no existan valores negativos
        for i, elemento in enumerate(args):
            if isinstance(elemento, (int, float)) and elemento < 0:
                raise ValueError(f"Error: No se aceptan valores negativos como: {elemento} en los argumentos de la función"
                 f" {funcion_a_decorar.__name__}, Valor incorrecto en posición {i}: {elemento}")
        
        print()
        resultado = funcion_a_decorar(*args, **kwargs)
        
        print(f'VALIDACIÓN exitosa de los datos de la función {funcion_a_decorar.__name__}')
        return resultado
    
    return funcion_interna_que_procesa_funcion_a_decorar_y_adiciona_comportamientos


def tiempo_de_procesamiento(funcion_a_decorar):
    
    @wraps(funcion_a_decorar)
    def funcion_interna_que_procesa_funcion_a_decorar_y_adiciona_comportamientos(*args_funcion_a_decorar, **kwargs_funcion_a_decorar):
        tiempo_inicial = time.time()
        print(f'Iniciando CALCULO DE TIEMPO DE PROCESAMIENTO para la función {funcion_a_decorar.__name__}')
        print()

        resultado = funcion_a_decorar(*args_funcion_a_decorar, **kwargs_funcion_a_decorar)
        time.sleep(1)

        tiempo_final = time.time()
        tiempo_total = round(tiempo_final - tiempo_inicial, 5)

        print(f'TIEMPO TOTAL DE PROCESAMIENTO DE LA FUNCIÓN {funcion_a_decorar.__name__}: {tiempo_total} segundos')
        print()
        return resultado
    
    return funcion_interna_que_procesa_funcion_a_decorar_y_adiciona_comportamientos

import random

def generar_notificacion(funcion_a_decorar):
    
    @wraps(funcion_a_decorar)
    def funcion_interna_que_procesa_funcion_a_decorar_y_adiciona_comportamientos(*args_funcion_a_decorar, **kwargs_funcion_a_decorar):
        print(f'Se inició la GENERACION DE NOTIFICACION de la función {funcion_a_decorar.__name__}')
        print()
        
        resultado = funcion_a_decorar(*args_funcion_a_decorar, **kwargs_funcion_a_decorar)
        print()
        
        estados = ['Exitoso', 'Fallido', 'en Espera']
        for clase in INotificacion.__subclasses__():
            estado = random.choice(estados)
            instancia = clase()
            instancia.crear_notificacion(funcion=funcion_a_decorar.__name__, descripcion=estado)
            
        print(f'NOTIFICACION completada de la función {funcion_a_decorar.__name__}')
        print()
        return resultado
    
    return funcion_interna_que_procesa_funcion_a_decorar_y_adiciona_comportamientos

def formatear_respuesta(funcion_a_decorar):
    
    @wraps(funcion_a_decorar)
    def funcion_interna_que_procesa_funcion_a_decorar_y_adiciona_comportamientos(*args_funcion_a_decorar, **kwargs_funcion_a_decorar):
        print(f'Se inició el FORMATEO de los datos de la función {funcion_a_decorar.__name__}')
        print()
        
        resultado = funcion_a_decorar(*args_funcion_a_decorar, **kwargs_funcion_a_decorar)
        casting = round(float(resultado), 2)
        print()
        
        print(f'FORMATEO exitoso del resultado de la función {funcion_a_decorar.__name__}')
        return casting

    return funcion_interna_que_procesa_funcion_a_decorar_y_adiciona_comportamientos

- Un decorador es una función que toma otra función como parámetro.
- Retorna una nueva función.
- La nueva función generalmente realiza algún procesamiento adicional antes o después de llamar a la función original.
- Los argumentos de la función original se pasan a la nueva función decorada.
- El resultado de la nueva función se devuelve.

# DECORADOR USO CLASICO

In [7]:
valores = [1.89, 2.23456, 7, 13]

def sumar(*args):
    print('Inicio Paso funcion SUMA :', args)
    print()
    return sum(args)

## USO 1

In [9]:
 tiempo_de_procesamiento(sumar)(*valores)

Iniciando CALCULO DE TIEMPO DE PROCESAMIENTO para la función sumar

Inicio Paso funcion SUMA : (1.89, 2.23456, 7, 13)

TIEMPO TOTAL DE PROCESAMIENTO DE LA FUNCIÓN sumar: 1.00702 segundos



24.12456

## USO 2

In [4]:
funcion_decorada = tiempo_de_procesamiento(sumar)
funcion_decorada(*valores)

Iniciando CALCULO DE TIEMPO DE PROCESAMIENTO para la función sumar

Inicio Paso funcion SUMA : (1.89, 2.23456, 7, 13)

TIEMPO TOTAL DE PROCESAMIENTO DE LA FUNCIÓN sumar: 1.00137 segundos



24.12456

# DECORANDO UN METODO DIRECTAMENTE CON @

In [5]:
@tiempo_de_procesamiento
@generar_notificacion
@formatear_respuesta
@validar_datos
def sumar(*args):
    print('Inicio Paso funcion SUMA :', args)
    print()
    return sum(args)
    
sumar(1.89, 2.23456, 7, 13)

Iniciando CALCULO DE TIEMPO DE PROCESAMIENTO para la función sumar

Se inició la GENERACION DE NOTIFICACION de la función sumar

Se inició el FORMATEO de los datos de la función sumar

Se inició la VALIDACIÓN de los datos de la función sumar

Inicio Paso funcion SUMA : (1.89, 2.23456, 7, 13)

VALIDACIÓN exitosa de los datos de la función sumar

FORMATEO exitoso del resultado de la función sumar

Notificacion enviada al Slack, El proceso de la funcion sumar fue en Espera!!!
Notificacion enviada al Jira, El proceso de la funcion sumar fue Fallido!!!
Notificacion enviada al Email, El proceso de la funcion sumar fue en Espera!!!
Notificacion enviada al Cloudwatch, El proceso de la funcion sumar fue en Espera!!!
Notificacion enviada al DataDog, El proceso de la funcion sumar fue Fallido!!!
Notificacion enviada al Log, El proceso de la funcion sumar fue Exitoso!!!
Notificacion enviada al Grafana, El proceso de la funcion sumar fue en Espera!!!
NOTIFICACION completada de la función sumar

TIE

24.12

# DECORAR UN DECORADOR

In [41]:
def decorador_de_decoradores(funcion_decoradora):
    
    def nombre_funcion_decoradora(funcion_a_decorar):
        
        def funcion_interna_que_procesa_funcion_a_decorar_y_adiciona_comportamientos(*args_funcion_a_decorar, **kwargs_funcion_a_decorar):
            
            print(f'Inicio Paso decorador_de_decoradores parametros:', args_funcion_a_decorar, kwargs_funcion_a_decorar)
            print('decorador_de_decoradores va a llamar al decorador :', funcion_decoradora.__name__)
            print('decorador_de_decoradores va a llamar a la funcion :', funcion_a_decorar.__name__)
            print()
            
            resultado = funcion_decoradora(funcion_a_decorar)(*args_funcion_a_decorar, **kwargs_funcion_a_decorar)
            print(f'Finalizo Paso decorador_de_decoradores con resultado :', resultado)
            print()
            return resultado
        
        return funcion_interna_que_procesa_funcion_a_decorar_y_adiciona_comportamientos
    
    return nombre_funcion_decoradora

@decorador_de_decoradores
def duplicar(funcion_a_procesar):
    def wrapper(*args, **kwargs):
        print(f'Inicio Paso funcion_decoradora parametros:', args, kwargs)
        print('funcion_decoradora va a llamar a la funcion :', funcion_a_procesar.__name__)
        print()
        
        resultado = funcion_a_procesar(*args, **kwargs) * 2
        print(f'Finalizo Paso funcion_decoradora  con resultado :', resultado)
        print()
        return resultado
    return wrapper

@duplicar
def sumar(*args):
    print('Inicio Paso funcion SUMA :', args)
    print()
    return sum(args)

resultado = sumar(4, 5, 6, 7)
print('Resultado final:', resultado)

Inicio Paso decorador_de_decoradores parametros: (4, 5, 6, 7) {}
decorador_de_decoradores va a llamar al decorador : duplicar
decorador_de_decoradores va a llamar a la funcion : sumar

Inicio Paso funcion_decoradora parametros: (4, 5, 6, 7) {}
funcion_decoradora va a llamar a la funcion : sumar

Inicio Paso funcion SUMA : (4, 5, 6, 7)

Finalizo Paso funcion_decoradora  con resultado : 44

Finalizo Paso decorador_de_decoradores con resultado : 44

Resultado final: 44


# DECORADOR CON ARGUMENTOS

In [45]:
def decorador_con_argumentos(*argumentos_decorador, **llave_valor_argumentos_decorador):

    def procesar_decorador(funcion_entrada):
        
        @wraps(funcion_entrada)
        def procesar_funcion(*argumentos_funcion, **llave_valor_argumentos_funcion):
            
            print("Argumentos decorador ", argumentos_decorador)
            print("Argumentos funcion ", argumentos_funcion)

            print("Argumentos llave valor decorador ", llave_valor_argumentos_decorador)
            print("Argumentos llave valor funcion ", llave_valor_argumentos_funcion)
            
            total = funcion_entrada(*argumentos_funcion, **llave_valor_argumentos_funcion) * 3
            
            print("resultado de la funcion TRIPLICADA ", total)
            return chr(total) + chr(total + 1) + chr(total) + f' {ord("S")} {ord("0")} {ord("S")}'
        
        return procesar_funcion
    return procesar_decorador

In [46]:
@decorador_con_argumentos(5,7,9,13, parametro_1 = "decorador con parametros en el decorador")
def funcion_decorada(*args, **kwargs):
    """
    funcion con argumentos con decorador con argumentos
    """
    
    return sum(args)

funcion_decorada(*[i for i in range(8)], **{chr(i + 35):i for i in range(10)})

Argumentos decorador  (5, 7, 9, 13)
Argumentos funcion  (0, 1, 2, 3, 4, 5, 6, 7)
Argumentos llave valor decorador  {'parametro_1': 'decorador con parametros en el decorador'}
Argumentos llave valor funcion  {'#': 0, '$': 1, '%': 2, '&': 3, "'": 4, '(': 5, ')': 6, '*': 7, '+': 8, ',': 9}
resultado de la funcion TRIPLICADA  84


'TUT 83 48 83'

# DECORANDO UN METODO DE UNA CLASE
### NO FUNCIONA CON HERENCIA

In [ ]:
class ICalculo:
    def sumar(self):
        pass
    
    @tiempo_de_procesamiento
    @generar_notificacion
    @formatear_respuesta
    @validar_datos
    def promedio(self):
        pass
    
class Calculos(ICalculo):
    def __init__(self, *args, **kwargs):
        print('Inicio de la Clase Calculos')
        self.__args = args
        
    @tiempo_de_procesamiento
    @generar_notificacion
    @formatear_respuesta
    @validar_datos
    def sumar(self):
        print('Inicio Paso funcion SUMA :', self.__args)
        print()
        return sum(self.__args)


    def promedio(self):
        print('Inicio Paso funcion PROMEDIO :', self.__args)
        print()
        return sum(self.__args)/len(self.__args)
    
calculos = Calculos(1.89, 2.23456, 7, 13)
suma = calculos.sumar()
print('Resultado final Funcion SUMA:', suma)
print()
print('---------------------------------------------')
print()
promedio = calculos.promedio()
print('Resultado final Funcion PROMEDIO:', promedio)

In [ ]:
class Otro(Calculos):
    def __init__(self, *args, **kwargs):
        print('Inicio de la Clase Calculos')
        self.__args = args

calculos = Otro(1.89, 2.23456, 7, 13)
suma = calculos.sumar()